In [1]:
!pip install kafka-python

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
     |████████████████████████████████| 246 kB 1.3 MB/s eta 0:00:01


In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [85]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.50 seconds
took 0.50 seconds


In [2]:
import pandas as pd

In [10]:
!wget -nc https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

File ‘green_tripdata_2019-10.csv.gz’ already there; not retrieving.



In [3]:
df_green = pd.read_csv("green_tripdata_2019-10.csv.gz", usecols=[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
])

In [14]:
df_green.head

<bound method NDFrame.head of        lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  DOLocationID  \
0       2019-10-01 00:26:02   2019-10-01 00:39:58           112           196   
1       2019-10-01 00:18:11   2019-10-01 00:22:38            43           263   
2       2019-10-01 00:09:31   2019-10-01 00:24:47           255           228   
3       2019-10-01 00:37:40   2019-10-01 00:41:49           181           181   
4       2019-10-01 00:08:13   2019-10-01 00:17:56            97           188   
...                     ...                   ...           ...           ...   
476381  2019-10-31 23:30:00   2019-11-01 00:00:00            65           102   
476382  2019-10-31 23:03:00   2019-10-31 23:24:00           129           136   
476383  2019-10-31 23:02:00   2019-10-31 23:23:00            61           222   
476384  2019-10-31 23:42:00   2019-10-31 23:56:00            76            39   
476385  2019-10-31 23:23:00   2019-10-31 23:56:00            56           215  

In [11]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    # print(row_dict)

    producer.send(topic_name, value=row_dict)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |17749|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |15949|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |14064|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |12843|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |11938|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |11539|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |10847|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |7913 |
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |7544 |
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |7318 |
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |7299 |
|{2024-

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |21153|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |19026|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |16880|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |15135|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |14198|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |13834|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |13022|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |9370 |
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |9023 |
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |8778 |
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |8604 |
|{2024-

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |24410|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |21768|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |19356|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |17484|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |16120|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |15712|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |15079|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |10851|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |10284|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |10126|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |9711 |
|{2024-

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |26815|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |23883|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |21291|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |19413|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |17964|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |17456|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |16726|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |12052|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |11445|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |11183|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |10905|
|{2024-

-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |29257|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |26110|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |23309|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |21051|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |19823|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |19272|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |18107|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |13106|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |12645|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |12177|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |11981|
|{2024-

-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |31908|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |28379|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |25330|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |22835|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |21529|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |20813|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |19736|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |14198|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |13713|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |13186|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |12940|
|{2024-

-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |34488|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |30655|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |27478|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |24737|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |23295|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |22508|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |21361|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |15512|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |14840|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |14331|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |14018|
|{2024-

took 68.36 seconds


-------------------------------------------
Batch: 8
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |35123|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |31436|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |27890|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |25369|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |23658|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |22867|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |21550|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |15722|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |15005|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |14531|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |14384|
|{2024-

-------------------------------------------
Batch: 9
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|74          |35482|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|42          |31884|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|41          |28122|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|75          |25680|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|129         |23860|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|7           |23066|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|166         |21690|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|236         |15826|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|223         |15084|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|238         |14636|
|{2024-03-14 20:40:00, 2024-03-14 20:45:00}|82          |14584|
|{2024-

In [ ]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

In [5]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [8]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/14 00:34:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b897b0e1-7035-4d8b-9b11-5aa057def5ec. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 00:34:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


[Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 14, 0, 31, 14, 245000), timestampType=0)]
Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 14, 0, 31, 14, 245000), timestampType=0)
[Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)]
Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:

In [9]:
query.stop()

In [6]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [7]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [12]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/03/14 00:35:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cbcf1b32-5485-4b3e-97f0-56368ba88551. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 00:35:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [13]:
query.stop()

In [26]:
popular_destinations = green_stream.withColumn("timestamp", F.current_timestamp()).groupBy(F.window(F.col("timestamp"), "5 minutes"), F.col("DOLocationID")).count().orderBy(F.desc('count'))

In [27]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

24/03/14 21:31:24 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6a218736-acec-4647-b974-450034e17d1c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/14 21:31:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+------------+-----+
|window|DOLocationID|count|
+------+------------+-----+
+------+------------+-----+



In [ ]:
query.stop()

In [28]:
topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    # print(row_dict)

    producer.send(topic_name, value=row_dict)

producer.flush()

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |4    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |4    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|188         |4    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|168         |3    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |2    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |2    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|92          |2    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|160         |2    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|49          |2    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|37          |2    |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|226         |2    |
|{2024-

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |2160 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |1755 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |1631 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |1558 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |1487 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |1182 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |1149 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |945  |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |804  |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|95          |765  |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|244         |756  |
|{2024-

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |4240 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |3806 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |3473 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |3062 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |2777 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |2745 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |2707 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |1854 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |1816 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |1777 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |1603 |
|{2024-

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |6822 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |5961 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |5432 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |4733 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |4307 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |4301 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |4288 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |2980 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |2846 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |2807 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |2532 |
|{2024-

-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |9077 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |7942 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |7231 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |6575 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |6034 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |5923 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |5882 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |4140 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |3904 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |3865 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |3613 |
|{2024-

-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |11440|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |10114|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |9189 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |8127 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |7876 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |7707 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |7193 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |5157 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |5086 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |4834 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |4681 |
|{2024-

-------------------------------------------
Batch: 7
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |13947|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |12195|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |11025|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |9874 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |9438 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |9113 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |8779 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |6197 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |6095 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |5789 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |5527 |
|{2024-

-------------------------------------------
Batch: 8
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |16444|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |14403|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |13153|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |11717|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |11130|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |10758|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |10356|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |7457 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |7149 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |6844 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |6604 |
|{2024-

-------------------------------------------
Batch: 9
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |17261|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |15325|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |13742|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |12416|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |11661|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |11265|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |10664|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |7782 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |7433 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |7180 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |7013 |
|{2024-

-------------------------------------------
Batch: 10
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|74          |17741|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|42          |15942|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|41          |14061|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|75          |12840|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|129         |11930|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|7           |11533|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|166         |10845|
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|236         |7913 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|223         |7542 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|238         |7318 |
|{2024-03-14 21:30:00, 2024-03-14 21:35:00}|82          |7292 |
|{2024

In [30]:
query.stop()

In [29]:
len(df_green[df_green['DOLocationID']==74])

17741